## Finetuning Event Detection on TAK

In [1]:
import pandas as pd
import phospho 
from dotenv import load_dotenv

In [2]:
load_dotenv()

phospho.init()
tasks_df = phospho.tasks_df(limit=10_000)
tasks_df.set_index("task_id", inplace=True)
tasks_df

,task_input,task_output,task_metadata,task_eval,task_eval_source,task_eval_at,task_created_at,session_id,session_length,event_name,event_created_at
task_id,,,,,,,,,,,
df4eaba05f16454d824d7bbc4c495fbd,Combien y a t il de camions en Europe ?,"En 2024, il y a plus de 6,5 millions de camion...","{'version_id': 'gpt-4', 'client_created_at': 1...",failure,owner,2024-06-10 11:43:58,2024-05-24 18:04:03,Sv6NeyX-the-tak,1,None,NaT
8e99109451be48e5a76ca4b262707b58,Que peux tu faire ?,"Je suis Tak, un expert en recherche web créé p...","{'version_id': 'gpt-4', 'client_created_at': 1...",failure,owner,2024-06-10 11:44:00,2024-05-24 18:03:23,z5qgRtM-the-tak,1,phospho,2024-05-25 12:05:49
1e726a58c8254bb8aa6a0c205eb5c7ed,Dernières nouvelles sur l'IA en France,Voici les dernières nouvelles sur l'intelligen...,"{'version_id': 'gpt-4', 'client_created_at': 1...",success,phospho-4,2024-05-24 18:11:08,2024-05-24 17:13:57,LZVP3cV-the-tak,1,news,2024-05-24 19:02:12
1e726a58c8254bb8aa6a0c205eb5c7ed,Dernières nouvelles sur l'IA en France,Voici les dernières nouvelles sur l'intelligen...,"{'version_id': 'gpt-4', 'client_created_at': 1...",success,phospho-4,2024-05-24 18:11:08,2024-05-24 17:13:57,LZVP3cV-the-tak,1,AI,2024-05-25 12:08:53
fdf732ca4c2f47f681e0730c6a41b69a,Écris moi un poème sur les robots,"Bien sûr, voici un poème sur les robots :\n\n-...","{'version_id': 'gpt-4', 'client_created_at': 1...",success,phospho-4,2024-05-24 18:11:09,2024-05-24 17:13:33,DVP9VqE-the-tak,1,None,NaT
...,...,...,...,...,...,...,...,...,...,...,...
41248135de364ac294cfb0393f5f67bf,how to abolish capitalism ?,None,"{'client_created_at': 1715288110, 'last_update...",success,owner,2024-05-25 07:17:44,2024-05-09 20:55:10,SAO979YT1D,1,None,NaT
99275f11cd6148079a3600ec0231363f,How to use tqm in python?,None,"{'client_created_at': 1715269690, 'last_update...",success,owner,2024-05-25 07:17:45,2024-05-09 15:48:10,J8SGV27O91,1,code,2024-05-24 19:03:32
573a58cdfa9c4f4c8f00a4a46e2dc5bf,Write me a poem about robots,None,"{'client_created_at': 1715269012, 'last_update...",success,owner,2024-05-25 07:32:01,2024-05-09 15:36:52,GLK6WEFS10,1,None,NaT


In [3]:
new_df = tasks_df.dropna(subset=["task_input", "task_output"]).copy()
new_df["text"] = new_df[['task_input', 'task_output']].agg(' '.join, axis=1)

event_df = pd.get_dummies(tasks_df['event_name'])
event_df = event_df.astype(int)
new_df = new_df[['text']].merge(event_df, left_index=True, right_index=True)

new_df = new_df.drop_duplicates()
new_df = new_df.groupby(by=["task_id", "text"]).sum().reset_index()
new_df = new_df.set_index("task_id")

new_df

,text,AI,code,news,penetration,phospho
task_id,,,,,,
00362cd6be974fffacc48b4690481216,que faire a new york ce week end? Voici quelq...,0,0,0,0,0
007452c5b9d242f48254fcf348a96fa6,"Réécris la chanson Gigi l'amoroso, en décrivan...",0,0,0,0,0
0122446bf08645b9971e1ebf7cfa942f,this doesn't create a nested structure You're ...,0,1,0,0,0
01ccbfd37d014d6d8d90ba39b2a202ed,la pratique du trail et de l'ultra trail provo...,0,0,0,0,0
01d135ffb53c4402b22684fa8abb6872,Quelle est la proportion d'étudiants ingénieur...,0,0,0,0,0
...,...,...,...,...,...,...
fee8ca4afcc2427f81b9052a4b3f73a0,Fais moi un plan détaillé d'un roadtrip de 4 m...,0,0,0,0,0
fef89e16e22943e8a6bef2acbc6cd957,quelle particularité as tu par rapport a chatG...,0,0,0,0,1
ff5acf321e27424ca1a6b408cd85c86e,make a judicial review for the massachusetts v...,0,0,0,0,0


In [6]:
new_df[["AI", "code", "news", "phospho"]].sum(axis=1).max()

3

In [7]:
new_df.to_csv(r"/Users/kyoto/git/phospho_project/new_tasks.csv", index=False)

# A first baseline model

In [4]:
from sklearn.model_selection import train_test_split

X = new_df['text']
y = new_df.drop(columns="text")
 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42)

In [5]:
import xgboost as xgb 
from hyperopt import fmin, tpe, hp, STATUS_OK
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score, precision_score, recall_score, multilabel_confusion_matrix, accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier,  RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import scipy.stats as stats
import numpy as np
import time 




#GridSearchCV param distribituion
param_dist_grid = {
    'max_depth': [3, 7, 8],
    'learning_rate': [0.1, 0.01, 0.001],
    'subsample': [0.6, 0.7, 0.8],
    'n_estimators': np.arange(50, 200, 10)
}


# best params found for GridSearchCV according to param_dist_grid
best_params_grid = { 
    'max_depth': 8,
    'learning_rate': 0.1,
    'subsample': 0.8
}


# RandomizedSearchCV param distribution (to find the best hyperparameters)
param_dist_rand = {
    'max_depth': stats.randint(3, 10),
    'learning_rate': stats.uniform(0.01, 0.1),
    'subsample': stats.uniform(0.5, 0.5),
    'n_estimators':stats.randint(50, 200)
}

# best params found for RandomizedSearchCV according to param_dist_rand
best_params_rand= {
    'learning_rate': 0.10383770662443578,
    'max_depth': 9,
    'n_estimators': 178,
    'subsample': 0.8946196605097921
}

# Bayesian Opti
space = {
    'max_depth': 2,
    'learning_rate': 0.09982190399764716,
    #'subsample': hp.uniform('subsample', 0.5, 1),
    'n_estimators': 444
}


# Define the objective function to minimize
def objective(params):

    vectorizer = TfidfVectorizer(min_df=3, max_df=0.8, ngram_range=(1, 2))
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    params = {
        'n_estimators': int(params['n_estimators']),
        'learning_rate': params['learning_rate'],
        'max_depth': int(params['max_depth'])

    }

    xgb_model = xgb.XGBClassifier(**params)
    xgb_model.fit(X_train_tfidf, y_train)
    y_pred = xgb_model.predict(X_test_tfidf)
    score = accuracy_score(y_test, y_pred)

    return {'loss': -score,  'status': STATUS_OK}


#best_params = fmin(fn=objective, space= space, algo=tpe.suggest, max_evals=100)
#print("Best set of hyperparameters: ", best_params)

# for bayesian opti: Best set of hyperparameters:  {'learning_rate': 0.13078277066523722, 'max_depth': 4.0, 'subsample': 0.7787875865151597} 4m
# poem Best set of hyperparameters:  {'learning_rate': 0.09982190399764716, 'max_depth': 2.0, 'subsample': 0.7059337091873221}


# model using best parameters found (best_params_random) for all methods
# xgb_model = xgb.XGBClassifier(
#     learning_rate=best_params_grid['learning_rate'],
#     max_depth=best_params_grid['max_depth'],
#     #n_estimators=best_params_random['n_estimators'],
#     subsample=best_params_grid['subsample']
# )

final_xgb_model = xgb.XGBClassifier(
    learning_rate= space['learning_rate'],
    max_depth=int(space['max_depth']),
    n_estimators = int(space['n_estimators']),
    #subsample=best_params['subsample'],
    use_label_encoder=False,
    eval_metric='logloss'
)



#class_weights = {0: 1, 1: 10}  # Class 0: 1, Class 1: 10
# Set "scale_pos_weight" based on class balance
#pos_weight = sum(class_weights.values()) / sum(class_weights.keys())



# Using tf-idf to transform text data into numerical values
# Transformed data is passed through RandomizedSearchCV to find the best hyperparameters
text_classifier = make_pipeline(
    TfidfVectorizer(min_df=3, max_df=0.8, ngram_range=(1, 2)),
    final_xgb_model
    #GridSearchCV(xgb_model, param_dist_grid, cv=4, scoring='accuracy')
   # RandomizedSearchCV(xgb_model, param_distributions=param_dist, n_iter=10, cv=4, scoring='accuracy')
   # MultiOutputClassifier(estimator= xgb_model) # scale_pos_weight=pos_weight, class_weight=class_weights (original)
)



text_classifier.fit(X_train, y_train)
#%time _ = text_classifier.fit(X_train, y_train)

# make predictions and calculate inference time
start_ts = time.time()

y_pred = text_classifier.predict(X_test)
end_ts = time.time()

print(f"Prediction Time [s]: {(end_ts-start_ts):.3f}")



# best_params = text_classifier.named_steps['gridsearchcv'].best_params_
# best_score = text_classifier.named_steps['gridsearchcv'].best_score_

# print("Best set of hyperparameters: ", best_params)
# print("Best score: ", best_score)

# Best params and best scores
# best_params = text_classifier.named_steps['randomizedsearchcv'].best_params_
# best_score = text_classifier.named_steps['randomizedsearchcv'].best_score_

# print("Best set of hyperparameters: ", best_params)
# print("Best score: ", best_score)


f1 = f1_score(y_test, y_pred, average="weighted")
print("F1 Score : {:.3f}".format(f1))

recall = recall_score(y_test, y_pred, average='weighted')
print("Recall Score : {:.3f}".format(recall))

precision = precision_score(y_test, y_pred, average='weighted')
print("Precision Score : {:.3f}".format(precision))

confusion = multilabel_confusion_matrix(y_test, y_pred)
print("Confusion Matrix : ", confusion)

Prediction Time [s]: 0.033
F1 Score : 0.810
Recall Score : 0.726
Precision Score : 0.953
Confusion Matrix :  [[[154   0]
  [  3  12]]

 [[157   1]
  [  5   6]]

 [[149   1]
  [  6  13]]

 [[165   0]
  [  3   1]]

 [[156   0]
  [  0  13]]]


In [16]:
preds_data = text_classifier.predict(new_df["text"])
preds = pd.DataFrame(
    columns=[f"{col}_pred" for col in y_train.columns], 
    data=preds_data,
    index=new_df.index,
)
preds_df = pd.concat([new_df, preds], axis=1)
preds_df

,text,AI,code,news,penetration,phospho,poem,AI_pred,code_pred,news_pred,penetration_pred,phospho_pred,poem_pred
task_id,,,,,,,,,,,,,
00362cd6be974fffacc48b4690481216,que faire a new york ce week end? Voici quelq...,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
007452c5b9d242f48254fcf348a96fa6,"Réécris la chanson Gigi l'amoroso, en décrivan...",0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0
0122446bf08645b9971e1ebf7cfa942f,this doesn't create a nested structure You're ...,0,1,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0
01ccbfd37d014d6d8d90ba39b2a202ed,la pratique du trail et de l'ultra trail provo...,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
01d135ffb53c4402b22684fa8abb6872,Quelle est la proportion d'étudiants ingénieur...,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
fee8ca4afcc2427f81b9052a4b3f73a0,Fais moi un plan détaillé d'un roadtrip de 4 m...,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
fef89e16e22943e8a6bef2acbc6cd957,quelle particularité as tu par rapport a chatG...,0,0,0,0,1,0,0.0,0.0,0.0,0.0,1.0,0.0
ff5acf321e27424ca1a6b408cd85c86e,make a judicial review for the massachusetts v...,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
is_bad_pred = (y_pred != y_test)['news']
is_bad_pred.name = "is_bad_pred_code"

bad_code_preds = preds_df.merge(is_bad_pred, left_index=True, right_index=True)
bad_code_preds

,text,AI,code,news,phospho,AI_pred,code_pred,news_pred,phospho_pred,is_bad_pred_code
task_id,,,,,,,,,,
02f59fc6aac34ca38e56474e97074432,transform this note in a Guideline process or ...,0,0,0,0,0,0,0,0,False
07b7c70f9422416e8546bece550056d0,Write me a poem about robots **Ode to the Meta...,0,0,0,0,0,0,0,0,False
08cdebdc08bb4064be7386d7e4983863,power point sur la bonne utilisation du médicm...,0,0,0,0,1,0,0,0,False
0beb2afc623041a2bd910c875bfc85f3,Genere une conversation de plus de 200 lignes ...,0,0,0,0,1,0,0,0,False
0c79d73292d740debc76d3f4f47542a4,Les chaussettes de l'archi duchesse sont elles...,0,0,0,0,0,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...
f96ba0b00ba14310a28d40e045506f1a,make a sentiment analysis Sure! Here is a sent...,1,0,0,0,0,0,0,0,False
fb8bb8cd3027488cbc3b0d4b1c554a4c,Ferme solaire de Crucey\t\tEure-et-Loir\tUrbas...,0,0,0,0,0,0,0,0,False
fdf732ca4c2f47f681e0730c6a41b69a,"Écris moi un poème sur les robots Bien sûr, vo...",0,0,0,0,0,0,0,0,False
